In [1]:
%pip install boto3
%pip install pandas
%pip install requests
%config Completer.use_jedi = False


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import csv
import pandas as pd
import requests
import base64
import boto3
from datetime import datetime
import os

## Initial configuration

In [41]:
path_to_configuration = '../ignored/jupyter_configuration.csv'
config = pd.read_csv(path_to_configuration)
config['key']

0                  api_path
1              project_path
2    path_to_bank_statement
3                 bank_type
Name: key, dtype: object

In [20]:
def get_value_from_config(key):
    return config.loc[config['key'] == key, 'value'].values[0]

In [21]:
def get_path_to_endpoint(endpoint):
    return get_config_value('api_path') + endpoint

In [37]:
path_to_bank_statement = get_value_from_config('path_to_bank_statement')
bank_type = get_value_from_config('bank_type')
test_endpoint = get_path_to_endpoint('health')

### Reading file content utils

In [23]:
text_encoding = 'Windows-1250'

def encode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64encode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

def decode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64decode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

## Read bank statement

In [28]:
bank_statement_content = Path(path_to_bank_statement).read_text(encoding=text_encoding)

In [29]:
bank_statement_content_base64 = encode_base64(bank_statement_content)

## Upload bank statement to AWS

### Set up filename

In [31]:
current_date = datetime.today().strftime('%Y-%m-%d')
bucket_name = 'expenses-counter-bank-statements'
filename = os.path.basename(path_to_bank_statement)
s3_filename = '{}/{}-{}'.format(current_date, bank_type, filename)
s3_filename

'2024-04-05/PKO_BP-pkobp.csv'

### Test connection

In [43]:
response = requests.get(test_endpoint)
response

<Response [200]>

### Call API Gateway for immediate analysis

In [33]:
response = requests.get(parse_bank_statements_endpoint, params=params, headers=headers)
response.status_code

NameError: name 'params' is not defined

In [32]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [25]:
s3 = boto3.client('s3')
result = None
with open(path_to_bank_statement, "rb") as file:
    s3.upload_fileobj(file, bucket_name, s3_filename)

### Map bank statements to transactions

In [102]:
params = {
    "accountStatement": bank_statement_content_base64,
    "bankType": bank_type
}

In [103]:
headers = {'content-type': 'application/json'}
response = requests.get(parse_bank_statements_endpoint, params=params, headers=headers)
response.status_code

414